# 🧠 Entangled Learning 2.2: Alternating Teacher-Student on Real Data


This notebook demonstrates **Entangled Learning 2.2**, where two models trained on different feature spaces take turns being the teacher and student. 

### Key Features:
- Uses real-world **Breast Cancer Wisconsin dataset**
- Teacher-Student roles alternate every epoch
- Combines **Cross-Entropy Loss** with **KL divergence**
- Visualizes loss and alignment dynamics
- Compares performance via ROC AUC scores


## 📊 Dataset & Preprocessing

In [ ]:

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess data
data = load_breast_cancer()
X = StandardScaler().fit_transform(data.data)
y = data.target
X_pca = PCA(n_components=10).fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test, X_pca_train, X_pca_test = train_test_split(
    X, y, X_pca, test_size=0.2, random_state=42
)


## 🧠 MLP Model Definitions

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def make_mlp(input_dim, units=[64, 32]):
    model = Sequential()
    for i, u in enumerate(units):
        if i == 0:
            model.add(Dense(u, activation='relu', input_shape=(input_dim,)))
        else:
            model.add(Dense(u, activation='relu'))
        model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    return model

mlp_A = make_mlp(X.shape[1])
mlp_B = make_mlp(X_pca.shape[1], [32])


## 🔁 Alternating Entangled Training

In [ ]:

from tensorflow.keras.losses import BinaryCrossentropy

optimizer_A = tf.keras.optimizers.Adam()
optimizer_B = tf.keras.optimizers.Adam()
bce = BinaryCrossentropy()

epochs = 10
batch_size = 32
history = {"loss_A": [], "loss_B": [], "kl_A": [], "kl_B": [], "lambda": []}

for epoch in range(epochs):
    λ = 0.05 * (epoch / (epochs - 1))
    history["lambda"].append(λ)
    idxs = np.random.permutation(len(X_train))
    X_A, X_B, y_batch = X_train[idxs], X_pca_train[idxs], y_train[idxs]
    loss_A_epoch, loss_B_epoch, kl_A_epoch, kl_B_epoch = [], [], [], []

    for i in range(0, len(X_A), batch_size):
        xb_A = X_A[i:i+batch_size]
        xb_B = X_B[i:i+batch_size]
        yb = y_batch[i:i+batch_size]

        with tf.GradientTape(persistent=True) as tape:
            out_A = mlp_A(xb_A, training=True)
            out_B = mlp_B(xb_B, training=True)
            out_A_np = out_A.numpy().flatten()
            out_B_np = out_B.numpy().flatten()

            ce_A = bce(yb, out_A)
            ce_B = bce(yb, out_B)
            kl_AB = tf.reduce_mean(out_B_np * tf.math.log(tf.clip_by_value(out_B_np / out_A_np, 1e-7, 1e7)))
            kl_BA = tf.reduce_mean(out_A_np * tf.math.log(tf.clip_by_value(out_A_np / out_B_np, 1e-7, 1e7)))

            if epoch % 2 == 0:
                total_A = ce_A
                total_B = ce_B + λ * kl_AB
            else:
                total_A = ce_A + λ * kl_BA
                total_B = ce_B

        grads_A = tape.gradient(total_A, mlp_A.trainable_weights)
        grads_B = tape.gradient(total_B, mlp_B.trainable_weights)
        optimizer_A.apply_gradients(zip(grads_A, mlp_A.trainable_weights))
        optimizer_B.apply_gradients(zip(grads_B, mlp_B.trainable_weights))

        loss_A_epoch.append(total_A.numpy())
        loss_B_epoch.append(total_B.numpy())
        kl_A_epoch.append(kl_BA.numpy())
        kl_B_epoch.append(kl_AB.numpy())

    history["loss_A"].append(np.mean(loss_A_epoch))
    history["loss_B"].append(np.mean(loss_B_epoch))
    history["kl_A"].append(np.mean(kl_A_epoch))
    history["kl_B"].append(np.mean(kl_B_epoch))


## 📈 Training Dynamics

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(history["loss_A"], label="MLP_A (original input) Loss")
plt.plot(history["loss_B"], label="MLP_B (PCA input) Loss")
plt.plot(history["kl_A"], '--', label="KL(A aligned to B)")
plt.plot(history["kl_B"], '--', label="KL(B aligned to A)")
plt.plot(history["lambda"], label="λ (Entanglement)", color='gray', linestyle=':')
plt.title("Entangled Learning 2.2 — Alternating Teacher-Student on Real Data")
plt.xlabel("Epoch")
plt.ylabel("Loss / KL")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## ✅ Evaluation & ROC Curves

In [ ]:

from sklearn.metrics import roc_curve, auc

y_pred_A = mlp_A.predict(X_test).flatten()
y_pred_B = mlp_B.predict(X_pca_test).flatten()

fpr_A, tpr_A, _ = roc_curve(y_test, y_pred_A)
fpr_B, tpr_B, _ = roc_curve(y_test, y_pred_B)
auc_A = auc(fpr_A, tpr_A)
auc_B = auc(fpr_B, tpr_B)

plt.figure()
plt.plot(fpr_A, tpr_A, label=f"MLP_A ROC (AUC={auc_A:.2f})")
plt.plot(fpr_B, tpr_B, label=f"MLP_B ROC (AUC={auc_B:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.title("ROC Curves for MLP_A and MLP_B")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()

print("MLP_A (original input) Test Accuracy:", np.mean((y_pred_A > 0.5) == y_test))
print("MLP_B (PCA input) Test Accuracy:", np.mean((y_pred_B > 0.5) == y_test))
